Rnaseq analysis, Report generated by Mathu Malar

In [ ]:
step1: Data Qc using Fastqc and Multiqc [ Note: Snakemake was used and script is attached below]

In [ ]:
SAMPLES = ["0H_Infected_Rep1", "0H_Infected_Rep2", "0H_Mock_Rep1","0H_Mock_Rep2","48H_Infected_Rep1","48H_Infected_Rep2","48H_Mock_Rep1","48H_Mock_Rep2","72H_Infected_Rep1","72H_Infected_Rep2","72H_Mock_Rep1","72H_Mock_Rep2"]

rule all:
    input:
        expand("/home/malarm/RNAseq/test/output/{sample}_R1_fastqc.zip", sample=SAMPLES),
        expand("/home/malarm/RNAseq/test/output/{sample}_R2_fastqc.zip", sample=SAMPLES),
        "/home/malarm/RNAseq/test/output/multiqc_report.html"



rule fastqc:
    input:
        "/home/malarm/RNAseq/{sample}_R1.fastq.gz",
        "/home/malarm/RNAseq/{sample}_R2.fastq.gz"
    output:
        "/home/malarm/RNAseq/test/output/{sample}_R1_fastqc.zip",
        "/home/malarm/RNAseq/test/output/{sample}_R2_fastqc.zip"
    shell:
        "fastqc {input} --outdir output"


rule multiqc:
    input:
        expand("/home/malarm/RNAseq/test/output/{sample}_R1_fastqc.zip", sample=SAMPLES),
        expand("/home/malarm/RNAseq/test/output/{sample}_R2_fastqc.zip", sample=SAMPLES)
    output:
        "/home/malarm/RNAseq/test/output/multiqc_report.html"
    shell:
        "multiqc -o output --filename multiqc_report {input}"


In [27]:
from IPython.display import display, HTML

display(HTML('<a href="https://drive.google.com/file/d/1Rx5JeE9olQsdejznDGf4mCbujOMB42O9/view?usp=drive_link">View HTML Document</a>'))

In [ ]:
Step2: Trimming using fastp [ Note: shell command attached here]

In [ ]:
for first in *_R1.fastq.gz; do
    second=$(echo "${first}" | sed 's/_R1/_R2/g')
    output1="${first%_R1.fastq.gz}_R1trimmed.fastq.gz"
    output2="${second%_R2.fastq.gz}_R2trimmed.fastq.gz"
    ../fastp -i "${first}" -I "${second}" -o "${output1}" -O "${output2}"
done


In [26]:
from IPython.display import display, HTML

display(HTML('<a href="https://drive.google.com/file/d/1nPp5-VLXlc8bfb3Mb9o5uqTsnbSMZ5WT/view?usp=drive_link">View HTML Document</a>'))


In [ ]:
Step3: Trimming with fastp worsens length distribution of the fastq sequences so need to give it a thinking does it need trimming? reports are attached below

In [25]:
from IPython.display import display, HTML

display(HTML('<a href="https://drive.google.com/file/d/19uEK_Sjsj9cu5448Eu8RsN6nm50SZ0bP/view?usp=drive_link">View HTML Document</a>'))


In [ ]:
checking overrepresented sequences with blast figured out that first one is from A. thaliana genome didn't remove it

In [ ]:
TTTCATATGAGATCGGACCAGCAGGCCATAGTGGTTCTGAGCGACGCTAA
TTCGTCAATAGCTGAAGCTTAGTGGCTTTTCTTGGAAGCAGGGTATCACT
CGGTCGTCAATAGCTGAAGCTTAGTGGCTTTTCTTGGAAGCAGGGTATCA

In [ ]:
for first in *_R1.fastq.gz; do
    second=$(echo "${first}" | sed 's/_R1/_R2/g')
    output1="${first%_R1.fastq.gz}_R1trimmed.fastq.gz"
    output2="${second%_R2.fastq.gz}_R2trimmed.fastq.gz"
    ../fastp -i "${first}" -I "${second}" -o "${output1}" --adapter_fasta /home/malarm/RNAseq/overrep_seq.fa  -O "${output2}"
done

In [10]:
from IPython.display import display, HTML

display(HTML('<https://drive.google.com/file/d/1HSgx7XGaGqbrSpF5oaqMdX70Q7xxJF9_/view?usp=sharing>View HTML Document</a>'))


Step4: Mapping the trimmed fastq file with Arab reference genome using STAR

In [9]:
%%bash

# Set the path to the STAR installation directory
STAR_HOME="/home/malarm/STAR-2.7.11b/bin/Linux_x86_64"

# Set the path to the STAR reference genome index
REFERENCE_GENOME="/home/malarm/RNAseq/trimmed/test"

# Set the input and output directories
INPUT_DIR="/home/malarm/RNAseq/trimmed/test"
OUTPUT_DIR="/home/malarm/RNAseq/trimmed/test"

# Build STAR index
$STAR_HOME/STAR --runMode genomeGenerate --genomeSAindexNbases 12 --genomeDir $REFERENCE_GENOME --genomeFastaFiles $REFERENCE_GENOME/TAIR10_Arabidopsis_Genome.fas

# Loop over all FASTQ files in the input directory
for sample in "$INPUT_DIR"/*_R1trimmed.fastq; do
    # Extract the sample name without extension
    sample_name=$(basename "$sample" _R1trimmed.fastq)

    # Run STAR on paired-end reads and output SAM file
    $STAR_HOME/STAR --genomeDir $REFERENCE_GENOME --readFilesIn "$INPUT_DIR"/"${sample_name}_R1trimmed.fastq" "$INPUT_DIR"/"${sample_name}_R2trimmed.fastq" --outFileNamePrefix "$OUTPUT_DIR"/"${sample_name}_aligned" --outSAMtype BAM SortedByCoordinate
done


	/home/malarm/STAR-2.7.11b/bin/Linux_x86_64/STAR --runMode genomeGenerate --genomeSAindexNbases 12 --genomeDir /home/malarm/RNAseq/trimmed/test --genomeFastaFiles /home/malarm/RNAseq/trimmed/test/TAIR10_Arabidopsis_Genome.fas
	STAR version: 2.7.11b   compiled: 2024-01-25T16:12:02-05:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Feb 14 23:09:57 ..... started STAR run
Feb 14 23:09:57 ... starting to generate Genome files
Feb 14 23:09:59 ... starting to sort Suffix Array. This may take a long time...
Feb 14 23:09:59 ... sorting Suffix Array chunks and saving them to disk...
Process is interrupted.


In [ ]:
Step5:counting the number of genes mapped using featurecounts , the output is count file

In [ ]:
featureCounts -t exon -g gene_id -a Araport11_GTF_genes_transposons.current.gtf -o count_out.txt *.bam